In [1]:
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt


# Filepaths
fp11 = "data/TweetSearch_2011.parquet"
fp12 = "data/TweetSearch_2012.parquet"
fp13 = "data/TweetSearch_2013.parquet"
fp14 = "data/TweetSearch_2014.parquet"
fp15 = "data/TweetSearch_2015.parquet"
fp16 = "data/TweetSearch_2016.parquet"
fp17 = "data/TweetSearch_2017.parquet"
fp18 = "data/TweetSearch_2018.parquet"
fp19 = "data/TweetSearch_2019.parquet"
fp20 = "data/TweetSearch_2020.parquet"
fp21 = "data/TweetSearch_2021.parquet"
fp22 = "data/TweetSearch_2022.parquet"

# Read the files
hashtags2011 = pd.read_parquet(fp11)
hashtags2012 = pd.read_parquet(fp12)
hashtags2013 = pd.read_parquet(fp13)
hashtags2014 = pd.read_parquet(fp14)
hashtags2015 = pd.read_parquet(fp15)
hashtags2016 = pd.read_parquet(fp16)
hashtags2017 = pd.read_parquet(fp17)
hashtags2018 = pd.read_parquet(fp18)
hashtags2019 = pd.read_parquet(fp19)
hashtags2020 = pd.read_parquet(fp20)
hashtags2021 = pd.read_parquet(fp21)
hashtags2022 = pd.read_parquet(fp22)

In [2]:
# joining year-based dataframes into two different dataframes (because of the explosion of number of tweets after 10/2020)

ht20to22 = pd.concat([hashtags2020, hashtags2021, hashtags2022])
ht11to19 = pd.concat([hashtags2011, hashtags2012, hashtags2013, hashtags2014, hashtags2015, hashtags2016, hashtags2017, hashtags2018, hashtags2019])

# only English language tweets are used because identified keywords are in English

ht20to22 = ht20to22.loc[ht20to22["lang"] == "en"]
#ht20to22

ht11to19 = ht11to19.loc[ht11to19["lang"] == "en"]
#ht11to19

#ht20to22.sample(n=50)["text"].to_list()
#ht11to19.sample(n=50)["text"].to_list()

## Function

In [3]:
# 'df' needs to be a dataframe of tweets that has a column "created_at" with timestamp starting wiht "yyy-mm-dd"
# returns a dataframe that contains all original columns with an added column called "count" which tells how many tweets were posted that day
# returned dataframe also includes column "day" that has string in format "yyyy-mm-dd"

def get_daily_tweet_counts(df):
    
    import pandas as pd
    
    # reset index to be able to use below for clause
    
    df = df.reset_index(drop=True)
    
    # makes a column "day" with string "yyy-mm-dd"
    
    days = []

    for i in range(len(df)):
        days.append(df["created_at"][i][:10])

    df["day"] = days
    
    # get "count" (number of tweets) for each day in the dataframe
    # change indexing so that there are two columns, "day" and "count"
    
    days_counts = pd.DataFrame(df["day"].value_counts())
    days_counts = days_counts.reset_index()
    days_counts = days_counts.rename(columns={"index" : "day", "day" : "count"}) 
    
    # merge days_counts with df to add corresponding count of daily tweets to each tweet
    
    ret = pd.merge(df, days_counts, how="left", on="day")
    
    return ret

## English

#### Filter 1: remove tweets only including general/Tigray conflict related keywords and no other gerd-specific keywords

In [4]:
# filter round 1: remove tweets including one or more of the words in 'list1_2' and none of the words in list 'list1'

list1 = ["gerd", "hydropower", "renaissance dam"]
list1_2 = ["tplf", "tigray", "unityforethiopia", "ethiopiaprevails", "handsoffethiopia"]

# choosing tweets that mention one of the irrelevant keywords

filter_gerd1 = ht20to22.loc[ht20to22["text"].str.lower().str.contains("|".join(list1_2), regex=True)]

# out of those (above), choose the tweets that (also) mention one of the relevant 

temp_filter1 = filter_gerd1.loc[filter_gerd1["text"].str.lower().str.contains("|".join(list1), regex=True)]

filter_gerd1 = pd.concat([temp_filter1, filter_gerd1]).drop_duplicates(keep=False)

# concat and drop_duplicated to perform 'ht20to22 - filter_gerd1' (remove values in filter_gerd1 from ht20to22)

filtered1 = pd.concat([ht20to22, filter_gerd1]).drop_duplicates(keep=False)

In [5]:
filter_gerd1.sample(n=20)["text"].to_list()

['Birtukan Mideksa helped rewrite the Ethiopian Constitution to nullify Tigray’s September 2020 regional election which was the bases for the #Tigray war of ethnic cleansing. She is one of the #TigrayGenocideAccomplices @NEBEthiopia https://t.co/otdiqvZpAv',
 'The 1983 famine took the lives of more than a million #Tigray|ans due to information &amp; access blockage. The @PMEthiopia is implementing both methods to repeat history while the whole world is watching in silence. #WorldHumanitarianDay2021 #AllowAccessToTigray @NorwayMFA @NorwayUN',
 'SR97 is no less than about a regime change in Ethiopia. The people of Ethiopia aspire democracy, peace and stability. Let democracy work in Ethiopia! #ForwardElectionEthiopia #UnityForEthiopia @EU_Commission @StateDept @SenateForeign @EU_Commission @AUC_MoussaFaki',
 '#UNSC \nwe urge you to drive:\n-Immediate and Complete withdrawal of Eritrean forces from #Tigray\n-Immediate and unfettered humanitarian access to Tigray.\n#UNSCActNow\n#EritreaOut

In [6]:
filtered1["text"].head(20).to_list()

['#Nile River Valley and the #Indus River Basin are the cradle and seat of civilization. People have lived and farmed in their fertile floodplains since inception of civilisation. \n\nLooking at the night map what is the major difference between Nile and Indus ? Will answer later! https://t.co/yKJo1X4eqg',
 '#Nile River Valley and the Indus River Basin are the cradle and seat of civilization. People have lived and farmed in their fertile floodplains for millennia. \n\nLooking at the night map what  is the major difference between Nile and Indus ? Like to know ! Will answer later',
 'Happy New Year! Taking swing lessons for the first time from Dad at the driving range on New Year’s Day...\n\n#drcarsonliu #bariatricsurgery #heartburnrelief #herniasurgery #gerd #hiatalhernia #hiatalherniarepair… https://t.co/T1zU0Epm17',
 "Many people have misconceptions about the relationship between #hiatalhernias and #GERD.  Let's discuss how these two painful conditions are connected: https://t.co/TVZ

In [8]:
print("tweets left, original amount, tweets left (%)")
print(len(filtered1), len(ht20to22), len(filtered1)/len(ht20to22))

tweets left, original amount, tweets left (%)
206528 5002288 0.041286707202784004


#### Filter 2A: using 'filtered1', remove tweets containing at least one medical keyword (2020-2022)

In [9]:
# filter round 2: remove tweets including at least one of the (medical) words in list 'list2'

list2 = ["hernia", "surgery", "heartburn", "chronicpain", "chronic pain", "chronicillness", "chronic illness", "esophagus", "esophageal", "disorder", "#west #nile virus", "gerdsymptom", "coughing", "healing"]

temp_filter2a = filtered1.loc[filtered1["text"].str.lower().str.contains("|".join(list2), regex=True)]

filtered2a = pd.concat([temp_filter2a, filtered1]).drop_duplicates(keep=False)

In [10]:
filtered2a.sample(n=20)["text"].to_list()

["@AbiyAhmedAli Ethiopia 100% owner, financier of #GERD, built on a river in Ethiopia, has the sole right to fill GERD at anytime. \n\nBut the big question and the fact of the matter is “Will Ethiopia and Egypt's Nile dam dispute lead to war? “\n\nI don’t think so... because, Ethiopia &amp; Egypt..1/2",
 "Africa, Europe, &amp; the Middle East can benefit from #GERD's ecofriendly electric power generation. #GERD4All #ItsAfricanDam @MfaEgypt @aftaburhan @MoZ3rban @SecBlinken @EU_Commission @antonioguterres https://t.co/EXPDqqWP4C",
 'African Solutions to African Problems! #Ethiopia #Sudan #Egypt #nile #GERD https://t.co/05Pe87yX4I',
 'It was a pleasure to meet H.E Ugo Astuto, Ambassador of European Union to India. We discussed about the recent declaration of Humanitarian truce by the gov’t of Ethiopia, GERD and related issues.@mfaethiopia #EthiopiaPrevails https://t.co/aUsNP5d51y',
 "The Grand Ethiopian Renaissance Dam (GERD)\n - Africa's biggest hydropower project\n - 4bn cost partly me

In [11]:
temp_filter2a.sample(n=15)["text"].to_list()

['Why did I get GERD?\n\n#gerd #chronicacidreflux #chronicacidrefluxdisease #gastroreflux #gastrorefluxdisease #gerdawareness #gerdawarenessweek  #heartburn https://t.co/S6mHZuXM6M',
 'This #GERDAwarenessWeek, learn which clinic-based exercise programs cost-effectively prevent falls and fractures in seniors on #PPI therapy for #GERD or other #GI disorders: https://t.co/Qkwhetdj9o  @JPCRR @IFFGD @AmCollegeGastro @AmerGastroAssn @AmerGeriatrics @MedscapeGastro',
 'A vibrating pill for constipation heads to the FDA after completing phase 3 study\n\n📲 https://t.co/aCBtQS6Mfg\n.\n.\n.\n#AnthonyStarpoliMD #NewYork #Poughkeepsie #Doctor #Gastroenterologist #Endosurgery #WeightLoss #Surgeon #IBS #Hemorrhoids #Heartburn #GERD https://t.co/BOzL0r9tB7',
 'Factors that can contribute to #GERD: Hiatal hernia, Obesity, Zollinger-Ellison syndrome, Hypercalcemia, Scleroderma, Visceroptosis.',
 'How to manage your weight without being hungry\nLearn Now: https://t.co/3p90htEdvA\n \n#AnthonyStarpoliMD #N

In [12]:
print("tweets left, original amount, tweets left (%), tweets reduced from last round (%), tweets removed (number)")
print(len(filtered2a), len(ht20to22), len(filtered2a)/len(ht20to22), len(filtered2a)/len(filtered1), len(temp_filter2a))

tweets left, original amount, tweets left (%), tweets reduced from last round (%), tweets removed (number)
205409 5002288 0.041063009566822226 0.9945818484660676 1119


#### Filter 2B: remove tweets containing at least one medical keyword (2011-2019)

In [13]:
# filter round 2B: remove tweets including at least one of the (medical) words in list 'list2'

temp_filter2b = ht11to19.loc[ht11to19["text"].str.lower().str.contains("|".join(list2), regex=True)]

filtered2b = pd.concat([temp_filter2b, ht11to19]).drop_duplicates(keep=False)

In [14]:
filtered2b.sample(n=50)["text"].to_list()

['Good morning Cairo 😍 Upgraded #Nile view from my hotel room this morning.\n.\n.\n.\n#Egypt #AncientHistory #egyptology #AncientEgyptian #NileView #Zamalek #TravelBlogger #travelblog #travel #Cairo #archaeologists… https://t.co/RVJgAT1dkK',
 'Azeb is an  active participant  of  donkey club in Mekele  primary school . #Tigray  #Donkeys #Ethiopia http://t.co/dbBLPBqun9',
 'I love Aswan 💕 \n😍\n\n#photooftheday \n#vscocam \n#nile \n#nature \n#view \n#vsco \n#photography… https://t.co/hMIPuG0jOg',
 '#Ethiopia controls the gates of #Nile and the life of Egyptians.\n#Egypt controls #Gaza gates and life of Gazans.\n\n#Grand #Ethiopian #renaissancedam \n#alnahda #dam #GERD https://t.co/F7AcjpDljn',
 '#Eritrea dusty+ashy skin color generation prefer obligation with #Tigray than #AfricanAmerican,#Caribbean, #Latino &amp; #SouthAsian in #ERITREA',
 'News: a round-up of highlights from the recent #NeuroGASTRO2019 meeting - https://t.co/nLe7RcysCK #free @esnm_eu #GERD #IBS #FGID #motility #pain htt

In [15]:
print("tweets left, original amount, tweets left (%), tweets removed (number)")
print(len(filtered2b), len(ht11to19), len(filtered2b)/len(ht11to19), len(temp_filter2b))

tweets left, original amount, tweets left (%), tweets removed (number)
107448 117818 0.9119828888624828 10370


#### Filter 3: remove tweets containing at least one music/tourism related keyword (2011-2022)

In [16]:
# concat results from 2A and 2B to be used in this step

filtered2 = pd.concat([filtered2a, filtered2b])

list3 = ["deathmetal", "death metal", "photography", "rodgers", "drummer", "spacevine", "cruise", "sunset", "guitar", "#travel", "#tourism", "metal", "#fashion", "bluenilediamond"]

temp_filter3 = filtered2.loc[filtered2["text"].str.lower().str.contains("|".join(list3), regex=True)]

filtered3 = pd.concat([temp_filter3, filtered2]).drop_duplicates(keep=False)
filtered3

,created_at,id,author_id,conversation_id,reply_settings,text,lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,...,user.public_metrics.following_count,user.public_metrics.tweet_count,user.public_metrics.listed_count,user.url,user.id,user.username,user.profile_image_url,user.created_at,user.location,user.description
1,2020-01-01T23:44:11.000Z,1212519922993111040,14450089,1212519922993111040,everyone,#Nile River Valley and the #Indus River Basin ...,en,0,0,2,...,2109,50523,38,https://t.co/B9zzZrNsFU,14450089,ilatif,https://pbs.twimg.com/profile_images/161942020...,2008-04-20T08:07:55.000Z,Paris,A strategic analyst keenly interested in our g...
2,2020-01-01T23:41:12.000Z,1212519169725472769,14450089,1212519169725472769,everyone,#Nile River Valley and the Indus River Basin a...,en,0,1,2,...,2109,50523,38,https://t.co/B9zzZrNsFU,14450089,ilatif,https://pbs.twimg.com/profile_images/161942020...,2008-04-20T08:07:55.000Z,Paris,A strategic analyst keenly interested in our g...
9,2020-01-01T20:39:50.000Z,1212473527426830338,1128019934422892544,1212473527426830338,everyone,"Happy New Year, Addis Hiking Family! \nWe had ...",en,1,1,9,...,40,204,4,None,1128019934422892544,addis_hiking,https://pbs.twimg.com/profile_images/156858721...,2019-05-13T19:31:24.000Z,Äthiopien,"It's all about adventure, admiring the beauty ..."
13,2020-01-01T16:01:45.000Z,1212403548048285696,451232651,1212403548048285696,everyone,The colorful Timkat festivity(Ethiopian Epipha...,en,0,0,5,...,1493,14909,35,https://t.co/lNhwn36qYJ,451232651,febewol7,https://pbs.twimg.com/profile_images/162193715...,2011-12-31T05:53:30.000Z,United States,#Ethiopia 24/7 | Music Enthusiast| 100% ETHIOP...
14,2020-01-01T13:09:03.000Z,1212360084589809665,451232651,1212360084589809665,everyone,#LandOfOrigins Love it!! https://t.co/zDcz25pfZg,en,0,0,1,...,1493,14909,35,https://t.co/lNhwn36qYJ,451232651,febewol7,https://pbs.twimg.com/profile_images/162193715...,2011-12-31T05:53:30.000Z,United States,#Ethiopia 24/7 | Music Enthusiast| 100% ETHIOP...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16909,2019-12-31T09:12:00.000Z,1211938040593231872,1180889598853029892,1211938040593231872,everyone,#Nile tradesman https://t.co/z5LSCHByqj,en,0,0,0,...,242,1356,0,None,1180889598853029892,DJappyPhotos,https://pbs.twimg.com/profile_images/121172121...,2019-10-06T16:56:45.000Z,None,Some of my photos for the world to see - pleas...
16910,2019-12-31T08:31:52.000Z,1211927943041044486,14337796,1211927943041044486,everyone,Kingdom by the Nile.\n.\n.\n.\n.\n.\n#🇪🇬 #medi...,en,0,0,1,...,820,1148,17,https://t.co/UfbSols4WE,14337796,rayshan,https://pbs.twimg.com/profile_images/378800000...,2008-04-09T01:09:26.000Z,San Francisco,Building infrastructure that builds the next g...
16934,2019-12-31T07:56:59.000Z,1211919162202820608,1081876641138143232,1211571560177385473,everyone,@Dawit99910437 @Emuye06 @speed_of_trust The pp...,en,0,0,0,...,821,20813,0,None,1081876641138143232,the_curious_2,https://pbs.twimg.com/profile_images/136089916...,2019-01-06T11:34:26.000Z,Vereinigte Staaten,Views are personal ...
16937,2019-12-31T06:03:00.000Z,1211890476284030976,3233487513,1211890476284030976,everyone,Such an exciting experience! An honor to have ...,en,0,0,0,...,58,679,10,None,3233487513,Chef_Yohanis,https://pbs.twimg.com/profile_images/595237017...,2015-05-04T14:30:47.000Z,None,None


In [17]:
print("tweets left, original amount, tweets left (%), tweets removed (number)")
print(len(filtered3), len(filtered2), len(filtered3)/len(filtered2), len(temp_filter3))

tweets left, original amount, tweets left (%), tweets removed (number)
291139 312857 0.930581703461965 21718


In [22]:
temp_filter3["text"].sample(n=10).to_list()

['RT @Astro_Ron: What eastern Med looked like yesterday just after sunset Tel Aviv #Amman #Nile #Cyprus #Istanbul  http://twitpic.com/64icak',
 '@djdavidcraig @nilerodgers @SandanceDubai CHIC #NILE RODGERS#the best of the best!!! Love you!!! from Asuncion /Paraguay!',
 'river nile cruise egypt photo hd image wallpaper http://t.co/pXsFEcNgb2 #river #nile #cruise #egypt #photo #hd #image #wallpaper',
 'One year ago #egypt #travel #sunset #discover #cruise #nile #nofilter https://t.co/4D6UsdDV7k https://t.co/AnOGDrrf5O',
 '#KodakAgfa Presents : A #Cairo pinkish #Sunset http://t.co/hQnolkAmbP #Nile #Photos #Photogallery',
 '#Travel diaries: Temples, ochre sand dunes, and the minarets of mosques during a #cruise down the #Nile, #Egypt http://t.co/Sz0tNz7g',
 'Hideous Divinity – Ages Die #HideousDivinity #BrutalDeathMetal #TechnicalDeathMetal FFO #HourOfPenance #Nile https://t.co/zceUTrzFEu',
 'Retweet if you love Brutal Fucking Death Metal!! \\mXm/\n#Origin #Cryptopsy #Carcass #CannibalCorp

#### Investigating results from rounds 1-3 & saving filtered dataframe (7.3.2023)

In [18]:
temp_df = get_daily_tweet_counts(filtered3)

In [19]:
temp_df.columns

Index(['created_at', 'id', 'author_id', 'conversation_id', 'reply_settings',
       'text', 'lang', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'possibly_sensitive',
       'referenced_tweets.id', 'referenced_tweets.type', 'in_reply_to_user_id',
       'user.public_metrics.followers_count',
       'user.public_metrics.following_count',
       'user.public_metrics.tweet_count', 'user.public_metrics.listed_count',
       'user.url', 'user.id', 'user.username', 'user.profile_image_url',
       'user.created_at', 'user.location', 'user.description', 'day', 'count'],
      dtype='object')

In [21]:
# save temp_df to data/filtered_data0703 as csv because parquet does not accept some objects in dataframe


#temp_df.to_csv("data/filter/en/filtered_data1703.csv")